# Introduction to FastAPI, asyncio, and HELAO

In [ ]:
!pip install uvicorn
!pip install fastapi
!pip install wait4it

# Imports 

The first step of this tutorial is to import some key python packages. They are as follows:

*   fastapi: allows easy construction of web APIs in python.
*   uvicorn: a package for hosting apps on servers.
*   requests: allows us to remotely interface with the functions of an app -- literally to make requests of a server.
*   asyncio: critical for all but the simplest server programming; enables a program to perform multiple concurrent actions in a single thread.
*   json: simple encoding of python lists and dictionaries as strings.
*   time: time.sleep() is used here to simulate any device functions that might take some time to complete.






In [4]:
#these are the key imports
from fastapi import FastAPI
import uvicorn
import requests
import asyncio
import json
import time
###########################

import nest_asyncio


# Basic Server Hosting

Below is a very simple example of a single function hosted on a server with fastAPI and uvicorn. Evaluate the cell below to start the server. This block of code has a few features that we will see with each of our fastapi-uvicorn servers:

*   at the top, the instantiation of a FastAPI() object to contain the web API;
*   below that, one or more normal python methods, with a body, return statement, and possibly one or more function parameters;
*   just above one or more of those functions, function decorators to add those functions to the app -- these call one of the HTTP request methods, such as "get" or "post", and specify a unique endpoint of the app as a string;
*   at the bottom, a call to uvicorn.run() specifying the URL and port on which to run the app. 

It's possible worth saying a few words about URLs and ports. You will generally have a few options for specifying an IP address as your URL. Firstly, you can use 127.0.0.1, which, called the loopback address, does not even expose your server to the local network. This is good for testing, as it is the same for every device (including whatever machine is remotely running this colab notebook), and cannot be accessed from another machine by mistake, and it is what we will use exclusively for this tutorial. Another option, when running servers locally, would be to use your device's local IP, assigned by your router. This usually begins with 192.168. It can be found on windows by running the command "ipconfig" in command prompt. Finally, you can expose your server to the web by using your device's global IP address.

Choice of port is a bit simpler -- it can basically be any integer below a certain size, though using 4 or 5 digits reduce the likelihood that you will try to use a port that is already being used by your computer.

In [ ]:
app = FastAPI() #define the app

@app.get("/hello") #function decorator adds function to the app
def hello(s: str):
    return {'greeting':f'hello {s}'}

nest_asyncio.apply() #necessary to make this app compatible with a jupyter notebook

uvicorn.run(app, host="127.0.0.1", port=5000) #uvicorn hosts the app as a server

After running that app, you may notice that the cell it is hosted in will not stop evaluating, and you are unable to do anything else. In the menu above, press Runtime -> Interrupt execution or press the cell's button to interrupt it, shutting down the server and allowing you to continue to use this notebook. When writing servers normally, we are content to launch them in a terminal window, and let them run as the main process of that terminal window while we run other code elsewhere. For the purposes of this tutorial, we wanted to keep everything in a single notebook. The code below will enable us to do that (how it works is not important) by starting each server as a new process. Evaluate the cell below and, for the rest of the tutorial only, run each server with

    start_api(app, host, port)

in place of 

    uvicorn.run(app, host, port)


In [5]:
#this code is necessary to make FastAPI and uvicorn work properly on colab
#it should be ignored
#adapted from https://github.com/David-Lor/FastAPI_LightningTalk-Notebook/blob/master/FastAPI.ipynb
import fastapi
from multiprocessing import Process
from wait4it import wait_for

server_processes = {}

def run(app: fastapi.applications.FastAPI, host: str, port: int):
    uvicorn.run(app=app,host=host,port=port)

def start_api(app: fastapi.applications.FastAPI, host: str, port: int):
    global server_processes
    if port in server_processes.keys():
        server_processes[port].terminate()
        del server_processes[port]
    server_processes[port] = Process(target=run,args=(app,host,port),daemon=True)
    server_processes[port].start()
    wait_for(port=port)


Ok, now this is the same app as above, but this time run in a way that is compatible with this colab notebook. Start the server and try to make some requests of it with the code below.

In [ ]:
app = FastAPI()

@app.get("/hello")
def hello(s: str):
    return {'greeting':f'hello {s}'}

start_api(app,"127.0.0.1",5000)

In [ ]:
#res = requests.get("http://127.0.0.1:5000/hello",params=dict(s="jack"))
#res
#res.json()

As you can see, the request takes the URL of the API endpoing as a string, and also accepts a dictionary of parameters to pass to the function. It returns a "response object", which by default shows the HTTP status code that the request got back from the server. Calling the ".json()" method of this response object gives you the actual return statement of the function.



# FastAPI for science

Now to demonstrate how we might use a server to host a scientific instrument. Below is an API for a simulated motor. It has two functions, one which moves the motor to a given coordinate and one which returns the current location of the motor. Turn it on and try moving it around a little bit!

In [ ]:
coordinates = [0,0,0] #3-dimensional position of an imaginary motorized stage

motor_app = FastAPI()

@motor_app.get("/move")
def move(loc: list):
  global coordinates
  if not all(isinstance(i,int) or isinstance(i,float) for i in loc): #loc must be a list of 3 numbers
    raise ValueError("coordinates were not properly specified")
  time.sleep(5) #function call to the API of the motor would go here, instead we just simulate the time it takes it to move.
  coordinates = loc
  return {'position': coordinates}

@motor_app.get("/getPos")
def getPos():
  #function call to the API of the motor would go here
  return {'position': coordinates}

start_api(motor_app,"127.0.0.1",5001)

In [ ]:
#try to control the motor here

You should notice pretty quickly that sending locations to the server is not working. That's because, when making the request, the python has to convert the parameters you send into HTML, and HTML has no equivalent to lists or dictionaries in python. In order to properly send these data types, we need to encode them in some way, and we usually do this with json. Try using json.dumps() to encode the list as a string when making the request, and add json.loads() within the motor function to decode it when it is received.

Now it's your turn to write a server from scratch!

Below I have a driver class for a simulated potentiostat, and the beginnings of a server to make it available to the network. Add the necessary functions to the server.

In [ ]:
datafuction

class potentiostat:
  
  def _init_():
    pass

  def measureVoltage(Vmin:float,Vmax:float,t:float):
    #
    return{'data':'I-V data'}

  def measureCurrents(Is:list,dt:float):
    #
    return{'data':'I-V data'}

  def voltage(): #get current voltage
    #
    return {'data': f'current voltage at {time.time()}'}

  def current(): #get current current
    #
    return {'data': f'current current at {time.time()}'}

potentiostat_app = FastAPI()

@potentiostat_app.on_event("startup")
async def start():
    p = potentiostat()



# Basic Asynchronous Programming

Ok, so now hopefully you understand the basics of writing and using servers with fastapi and uvicorn, and can see how these can be used to make the functions of experimental devices available on a network. Ultimately, we are interested in showing you how we script multiple servers together in HELAO, and that requires talking about some more advanced concepts that are central to writing web servers. Asynchronous programming can be tricky to get the hang of, and a full introduction is beyond the scope of this workshop, but we hope to familiarize you with the basics here.

There are two key words that you need to understand: "async" and "await". The keyword "async" is used as part of a function definiton to designate a function as asynchronous. The "await" keyword is then used to call asynchronous functions. The rules are that every function defined with the async keyword must be called with the await keyword, and every function that contains an await keyword must be defined with the async keyword.

But what does an asynchronous function actually do? It is hard to explain, but basically, when your code hits an "await" statement, it reads that as, "I can go do other things while waiting for this to complete." What "other things" means in this context is complicated -- the code isn't going to skip that line, for example -- but in the context of our servers, it means that the server is free to process other requests while one request is running.



This is tricky to describe without showing an example. So, to start, let's try to make the motor server from before asynchronous. A motor typically takes several seconds to move from one postion to another. With the code as we wrote it previously, the server will not respond to any requests received while the "motor" is in motion. The goal here is to rewrite the server so that is able to respond while the "motor" is moving. We don't want the server to allows a second user to change the motion of the "motor" once it has already started, but we do want to allows them to make the request, and notify them that the "motor" is not available.

In [ ]:
coordinates = [0,0,0] #3-dimensional position of an imaginary motorized stage

motor_app = FastAPI()

@motor_app.get("/move")
def move(loc: str):
  loc = json.loads(loc)
  if not all(isinstance(i,int) or isinstance(i,float) for i in loc): #loc must be a list of 3 numbers
    raise ValueError("coordinates were not properly specified")
    time.sleep(5) #function call to the API of the motor would go here, instead we just simulate the time it takes it to move.
  coordinates = loc
  return {'position': coordinates}

@motor_app.get("/getPos")
def getPos():
  #function call to the API of the motor would go here
  return {'position': coordinates}

start_api(motor_app,"127.0.0.1",5001)

Unfortunately, there is no easy way to make two requests at once from a colab notebook, so we can't test whether this server works as expected, but you should be able to see that it handles single requests exactly as it did before.

# The HELAO orchestrator

The server below is a simplified version of the orchestratator we use to run our experiments. Its function is to serve as a central point for all elements of an experiment. We send it lists of functions to perform, and it distributes those functions to the proper device servers and runs them one at a time. Overtop of that, we can then build in fancier features, such as automated data management, visualization, active learning, etc...



In [ ]:
#primitive HELAO-style orchestrator

orchestrator = FastAPI()

#tell the orchestrator where to find the servers it needs to communicate with.
url_table = {'motor':"http://127.0.0.1:5001",'potentiostat':"http://127.0.0.1:5003"}

#main endpoint of orchestrator -- receives experiments to be performed from user
@orchestrator.post("/addExperiment")
async def sendMeasurement(experiment: str):
    await experiment_queue.put(experiment)

#pulls data from the orchestrator from whatever experiments have been completed so far
@orchestrator.get("/getData")
def getData():
    global data
    return data
    
#infinite loop which runs as long as the server is up and performs experiments one at a time.
async def scheduler():
    while True:
        experiment = await experiment_queue.get()
        await doMeasurement(experiment)

#send the appropriate request to the appropriate server, and collect the results
async def doMeasurement(experiment: str):
    global loop,data,exp_count
    experiment = json.loads(experiment)
    data[f'experiment_{exp_count}'] = {}
    for item in experiment['soe']:
        server,action_str = item.split('/') #pull proper endpoint and parameter values out of the experiment dictionary
        params = experiment['params'][action_str]
        action = action_str.split('_')[0]
        url = url_table[server]
        data[f'experiment_{exp_count}'][action_str] = 'here'
        try:
            res = await loop.run_in_executor(None,lambda x: requests.get(x,params=params).json(),
                                             f"{url}/{action}") #perform experimental function
            data[f'experiment_{exp_count}'][action_str] = res #append data from function call to global database
        except Exception as e:
            data[f'experiment_{exp_count}'][action_str] = str(e)
    exp_count += 1

#on server startup, initialize some useful global objects
@orchestrator.on_event("startup")
def memory():
    global experiment_queue, task, loop, data, exp_count
    experiment_queue = asyncio.Queue()
    task = asyncio.create_task(scheduler())
    loop = asyncio.get_event_loop()
    data = {}
    exp_count = 0
    
start_api(orchestrator, "127.0.0.1", 5003) 

Here we can use the two dummy driver servers we have written to send fake experiments to the orchestrator. These experiments take the form of dictionaries with two defined keys. The first key, "soe", will contain a list of functions to perform, formatted 

In [ ]:
soe = ['motor/move_1','motor/move_2']
params = {'move_1':{'loc':json.dumps([5,5,5])},'move_2':{'loc':json.dumps([0,0,0])}}
requests.post("http://127.0.0.1:5003/addExperiment",params=dict(experiment=json.dumps({'soe':soe,'params':params})))

In [ ]:
requests.get("http://127.0.0.1:5003/getData",params=None).json()